# Compute Antarctic, depth integrated, cross slope heat transport, online terms

In [1]:
%matplotlib inline

import cosima_cookbook as cc
from cosima_cookbook import distributed as ccd
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import xarray as xr
import glob,os
import cmocean.cm as cmocean

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client

In [2]:
#client = Client(n_workers=4)
# >> dask-scheduler
# >> dask-worker tcp://10.0.64.9:8786 --memory-limit 4e9 --nprocs 6 --nthreads 1 --local-directory /local/g40/amh157
#client = Client('tcp://10.0.64.9:8786', local_dir='/local/g40/amh157')
client = Client()
client

Client Scheduler: tcp://127.0.0.1:43239 Dashboard: /proxy/34417/status,Cluster Workers: 7 Cores: 28 Memory: 137.44 GB


In [3]:
session = cc.database.create_session('/g/data/v45/akm157/jupyter_scripts/tides/ryf9091_tides.db')

In [4]:
year = '2095'
exp = '01deg_jra55v13_ryf9091'
#exp = '01deg_jra55v13_ryf9091_tides_fixed'

start_time='2095-01-01'
end_time='2095-12-31'
time_period = str(int(start_time[:4]))+'-'+str(int(end_time[:4]))

# reference density value:
rho_0 = 1035.0
# specific heat capacity of sea water:
cp = 3992.1
lat_range = slice(-90,-59)

isobath_depth = 1000

# pick a freezing point temp:
temp_freezing = -2.5

## Open contour data, extract lat/lon on contour

In [5]:
outfile = '/g/data/v45/akm157/model_data/access-om2/Antarctic_slope_contour_'+str(isobath_depth)+'m.npz'
data = np.load(outfile)
mask_y_transport = data['mask_y_transport']
mask_x_transport = data['mask_x_transport']
mask_y_transport_numbered = data['mask_y_transport_numbered']
mask_x_transport_numbered = data['mask_x_transport_numbered']

yt_ocean = cc.querying.getvar(exp,'yt_ocean',session,n=1)
yt_ocean = yt_ocean.sel(yt_ocean=lat_range)
yu_ocean = cc.querying.getvar(exp,'yu_ocean',session,n=1)
yu_ocean = yu_ocean.sel(yu_ocean=lat_range)
xt_ocean = cc.querying.getvar(exp,'xt_ocean',session,n=1)
xu_ocean = cc.querying.getvar(exp,'xu_ocean',session,n=1)

# convert isobath masks to data arrays, so we can multiply them later:
mask_x_transport = xr.DataArray(mask_x_transport, coords = [('yt_ocean', yt_ocean), ('xu_ocean', xu_ocean)])
mask_y_transport = xr.DataArray(mask_y_transport, coords = [('yu_ocean', yu_ocean), ('xt_ocean', xt_ocean)])
mask_x_transport_numbered = xr.DataArray(mask_x_transport_numbered, coords = [('yt_ocean', yt_ocean), ('xt_ocean', xt_ocean)])
mask_y_transport_numbered = xr.DataArray(mask_y_transport_numbered, coords = [('yt_ocean', yt_ocean), ('xt_ocean', xt_ocean)])

num_points = int(np.maximum(np.max(mask_y_transport_numbered),np.max(mask_x_transport_numbered)))

In [6]:
lat_along_contour = np.zeros((num_points))
lon_along_contour = np.zeros((num_points))

# locations for zonal transport:
x_indices_masked = mask_x_transport_numbered.stack().values
x_indices = np.sort(x_indices_masked[x_indices_masked>0])
for count in x_indices:
    count = int(count)
    jj = int(np.where(mask_x_transport_numbered==count)[0])
    ii = int(np.where(mask_x_transport_numbered==count)[1])   
    lon_along_contour[count-1] = xu_ocean[ii].values
    lat_along_contour[count-1] = mask_x_transport_numbered.yt_ocean[jj].values
    
# locations for meridional transport:
y_indices_masked = mask_y_transport_numbered.stack().values
y_indices = np.sort(y_indices_masked[y_indices_masked>0])
for count in y_indices:
    count = int(count)
    jj = np.where(mask_y_transport_numbered==count)[0]
    ii = np.where(mask_y_transport_numbered==count)[1]
    lon_along_contour[count-1] = mask_x_transport_numbered.xt_ocean[ii].values
    lat_along_contour[count-1] = yu_ocean[jj].values

In [7]:
lat_along_contour

array([-66.00806249, -66.00806249, -66.00806249, ..., -66.05030184,
       -66.05030184, -66.02918216])

## Compute heat transports calculated online

In [74]:
# Note temp_yflux_adv is also positioned on north centre edge of t-cell.
# temp_yflux_adv = cp*rho*dzt*dxt*v*temp

temp_yflux = cc.querying.getvar(exp,'temp_yflux_adv',session,start_time=start_time, end_time=end_time)
temp_xflux = cc.querying.getvar(exp,'temp_xflux_adv',session,start_time=start_time, end_time=end_time)

# select latitude range:
temp_yflux = temp_yflux.sel(yu_ocean=lat_range).sel(time=slice(start_time,end_time))
temp_xflux = temp_xflux.sel(yt_ocean=lat_range).sel(time=slice(start_time,end_time))

# time average and sum in depth:
temp_yflux = temp_yflux.mean('time').sum('st_ocean')
temp_xflux = temp_xflux.mean('time').sum('st_ocean')

temp_yflux = temp_yflux.load()
temp_xflux = temp_xflux.load()

Exception during reset or similar
Traceback (most recent call last):
  File "/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.04/lib/python3.7/site-packages/sqlalchemy/pool/base.py", line 693, in _finalize_fairy
    fairy._reset(pool)
  File "/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.04/lib/python3.7/site-packages/sqlalchemy/pool/base.py", line 880, in _reset
    pool._dialect.do_rollback(self)
  File "/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.04/lib/python3.7/site-packages/sqlalchemy/engine/default.py", line 540, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 23453619058496 and this is thread id 23450154944256.
Exception closing connection <sqlite3.Connection object at 0x15540694d490>
Traceback (most recent call last):
  File "/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.04/lib/python3.7/site-packages/sqlalchemy/po

In [91]:
# save a long term average of vhrho_nt and uhrho_et:

outpath = '/g/data/v45/akm157/model_data/access-om2/'+exp+'/Antarctic_cross_slope/uhrho_vhrho_'+time_period+'.nc'
# check if already exists:
if os.path.exists(outpath):
    average_transports = xr.open_dataset(outpath)
    # extract arrays from dataset:
    uhrho_et = average_transports.uhrho_et
    vhrho_nt = average_transports.vhrho_nt
else:
    vhrho_nt = cc.querying.getvar(exp,'vhrho_nt',session,start_time=start_time, end_time=end_time)
    uhrho_et = cc.querying.getvar(exp,'uhrho_et',session,start_time=start_time, end_time=end_time)

    vhrho_nt = vhrho_nt.sel(yt_ocean=lat_range).sel(time=slice(start_time,end_time))
    uhrho_et = uhrho_et.sel(yt_ocean=lat_range).sel(time=slice(start_time,end_time))

    vhrho_nt = vhrho_nt.mean('time')
    uhrho_et = uhrho_et.mean('time')

    outpath = '/g/data/v45/akm157/model_data/access-om2/'+exp+'/Antarctic_cross_slope/uhrho_vhrho_'+time_period+'.nc'
    ds = xr.Dataset({'vhrho_nt': vhrho_nt,'uhrho_et':uhrho_et})
    ds.to_netcdf(outpath)
    ds.close()

In [75]:
# subtract freezing point heat transport:
yt_ocean = cc.querying.getvar('01deg_jra55v13_ryf9091','yt_ocean',session,n=1)
dxu = cc.querying.getvar('01deg_jra55v13_ryf9091','dxu',session,n=1)
dyt = cc.querying.getvar(exp,'dyt',session,n=1)
# give dxu and dyt correct coordinates:
dxu.coords['nj'] = yt_ocean.values
dxu.coords['ni'] = xt_ocean['xt_ocean'].values
dxu = dxu.rename(({'ni':'xt_ocean', 'nj':'yt_ocean'}))
dyt.coords['nj'] = yt_ocean.values
dyt.coords['ni'] = xt_ocean['xt_ocean'].values
dyt = dyt.rename(({'ni':'xt_ocean', 'nj':'yt_ocean'}))
# select latitude range:
dxu = dxu.sel(yt_ocean=lat_range)
dyt = dyt.sel(yt_ocean=lat_range)

# Note that in newer mom5 versions this could also be done with ty_trans_int_z, 
# but there is a problem with this diagnostic in older runs, and even
# using ty_trans, there is a slight difference. Not sure why?

# Note vhrho_nt is v*dz*1035 and is positioned on north centre edge of t-cell.
# sum in depth:
vhrho_nt = vhrho_nt.sum('st_ocean')
uhrho_et = uhrho_et.sum('st_ocean')
# convert to transport:
vhrho_nt = vhrho_nt*dxu/rho_0
uhrho_et = uhrho_et*dyt/rho_0

# overwrite coords, so we can add the freezing point (with uhrho_et and vhrho_nt) without problems:
yu_ocean = cc.querying.getvar(exp,'yu_ocean',session,n=1)
yu_ocean = yu_ocean.sel(yu_ocean=lat_range)
vhrho_nt.coords['yt_ocean'] = yu_ocean.values
vhrho_nt = vhrho_nt.rename(({'yt_ocean':'yu_ocean'}))
uhrho_et.coords['xt_ocean'] = xu_ocean.values
uhrho_et = uhrho_et.rename(({'xt_ocean':'xu_ocean'}))

freezing_point_heat_trans_zonal = cp*rho_0*uhrho_et*temp_freezing
freezing_point_heat_trans_meridional = cp*rho_0*vhrho_nt*temp_freezing

# compare both ways:
temp_yflux = temp_yflux - freezing_point_heat_trans_meridional
temp_xflux = temp_xflux - freezing_point_heat_trans_zonal

temp_yflux = temp_yflux.load()
temp_xflux = temp_xflux.load()

# multiply by isobath contour masks:
temp_yflux_with_mask = temp_yflux*mask_y_transport
temp_xflux_with_mask = temp_xflux*mask_x_transport

## Extract heat transport values along isobath contour:

In [76]:
# multiply by mask to get correct direction into or out of isobath contour:
heat_trans_across_contour = np.zeros((num_points))

# locations for zonal transport, already calculated indices above:
for count in x_indices:
    count = int(count)
    jj = int(np.where(mask_x_transport_numbered==count)[0])
    ii = int(np.where(mask_x_transport_numbered==count)[1])
    heat_trans_across_contour[count-1] += temp_xflux_with_mask[jj,ii].values
    
# locations for meridional transport, already calculated indices above:
for count in y_indices:
    count = int(count)
    jj = int(np.where(mask_y_transport_numbered==count)[0])
    ii = int(np.where(mask_y_transport_numbered==count)[1])
    heat_trans_across_contour[count-1] += temp_yflux_with_mask[jj,ii].values

## Convert cross-slope heat transport from isobath coordinate to longitude coordinate

In [78]:
# convert to longitude coordinate and average into 3 degree longitude bins:

# in degrees:
bin_width = 3
bin_spacing = 0.25
lon_west = -280
lon_east = 80

# new coordinate and midpoints of longitude bins:
full_lon_coord = np.arange(lon_west,lon_east+bin_spacing,bin_spacing)
lon_bin_midpoints = np.arange(lon_west+bin_width/2,lon_east-bin_width/2,bin_spacing)
n_bin_edges = len(full_lon_coord)

# sum into longitude bins:
# need to be very careful of loops, we can't just mask over longitude values, but instead pick indices 
# on the isobath contour and sum continously along contour between defined indices.
# (i.e. lon_along_contour is not monotonic)
# find points on contour to define edges of longitude bins:
bin_edge_indices = np.zeros(n_bin_edges)
for lon_bin in range(n_bin_edges-1):
    # find first isobath point that has the right longitude:
    first_point = np.where(lon_along_contour>=full_lon_coord[lon_bin])[0][0]
    # then find all other isobath points with the same longitude as that first point:
    same_lon_points = np.where(lon_along_contour==lon_along_contour[first_point])[0]
    # we want the most southerly of these points on the same longitude line:
    bin_edge_indices[lon_bin] = same_lon_points[np.argmin(lat_along_contour[same_lon_points])]
    
# define east/west edges:
bin_edge_indices = bin_edge_indices.astype(int)
bin_edge_indices_west = bin_edge_indices[:-int(bin_width/bin_spacing)-1]
bin_edge_indices_east = bin_edge_indices[int(bin_width/bin_spacing):-1]
n_bins = len(bin_edge_indices_west)

# sum heat transport from isobath coord into new longitude coord:
cross_slope_heat_trans = np.zeros(n_bins)
for lon_bin in range(n_bins):
    heat_trans_this_bin = heat_trans_across_contour[bin_edge_indices_west[lon_bin]:bin_edge_indices_east[lon_bin]]
    cross_slope_heat_trans[lon_bin] = np.sum(heat_trans_this_bin)
    
# find average latitude of each bin, so we can plot back on the isobath:
lat_bin_midpoints = np.zeros(n_bins)
for lon_bin in range(n_bins):
    # find nearest isobath point:
    lon_index = np.where(lon_along_contour>=lon_bin_midpoints[lon_bin])[0][0]
    lat_bin_midpoints[lon_bin] = lat_along_contour[lon_index]

## Zonal heat convergence

In [79]:
# Need to make sure the zonal boundaries here match exactly with the zonal boundaries used for the 
# longitude averaging above, by using same bin_edge_indices.
# Just check if isobath point is on x or y grid. if x, sum up to this point from south.
# if on y grid, sum zonal transport on nearest u grid point to west.

zonal_heat_trans_west = np.zeros(n_bins)
for lon_bin in range(n_bins):
    # west limit:
    # reset these to False:
    on_x_grid = False
    on_y_grid = False
    # mask_x_transport_numbered etc indexing starts from 1 not 0, so add 1:
    isobath_west_index = int(bin_edge_indices_west[lon_bin]+1)
    # check if the point is on the x or y transport grid:
    if len(np.where(mask_x_transport_numbered==isobath_west_index)[0])>0:
        on_x_grid = True
        jj = int(np.where(mask_x_transport_numbered==isobath_west_index)[0])
        ii = int(np.where(mask_x_transport_numbered==isobath_west_index)[1])
    elif len(np.where(mask_y_transport_numbered==isobath_west_index)[0])>0:
        on_y_grid = True
        jj = int(np.where(mask_y_transport_numbered==isobath_west_index)[0])
        ii = int(np.where(mask_y_transport_numbered==isobath_west_index)[1])
    if on_x_grid == True:
        zonal_heat_trans_west[lon_bin] = np.sum(temp_xflux[:jj,ii])
    # in this case we want transport half a grid point to the west:
    elif on_y_grid == True:
        # careful if ii=0, then we need heat trans from lon=80, because at limit of zonal grid
        if ii==0:
            zonal_heat_trans_west[lon_bin] = np.sum(temp_xflux[:jj+1,-1])
        else:
            zonal_heat_trans_west[lon_bin] = np.sum(temp_xflux[:jj+1,ii-1])

zonal_heat_trans_east = np.zeros(n_bins)
for lon_bin in range(n_bins):
    # east limit:
    # reset these to False:
    on_x_grid = False
    on_y_grid = False
    # mask_x_transport_numbered etc indexing starts from 1 not 0, so add 1:
    isobath_east_index = int(bin_edge_indices_east[lon_bin]+1)
    # check if the point is on the x or y transport grid:
    if len(np.where(mask_x_transport_numbered==isobath_east_index)[0])>0:
        on_x_grid = True
        jj = int(np.where(mask_x_transport_numbered==isobath_east_index)[0])
        ii = int(np.where(mask_x_transport_numbered==isobath_east_index)[1])
    elif len(np.where(mask_y_transport_numbered==isobath_east_index)[0])>0:
        on_y_grid = True
        jj = int(np.where(mask_y_transport_numbered==isobath_east_index)[0])
        ii = int(np.where(mask_y_transport_numbered==isobath_east_index)[1])
    if on_x_grid == True:
        zonal_heat_trans_east[lon_bin] = np.sum(temp_xflux[:jj,ii])
    # in this case we want transport half a grid point to the west:
    elif on_y_grid == True:
        # wrap around to east side of grid:
        if ii==0:
            zonal_heat_trans_east[lon_bin] = np.sum(temp_xflux[:jj+1,-1])
        else:
            zonal_heat_trans_east[lon_bin] = np.sum(temp_xflux[:jj+1,ii-1])

zonal_convergence = zonal_heat_trans_east - zonal_heat_trans_west

## Save cross-slope and zonal convergence terms for this isobath

In [80]:
# convert to data arrays, so we can save as netcdf:
zonal_convergence = xr.DataArray(zonal_convergence, coords = [('lon_bin_midpoints', lon_bin_midpoints)])
cross_slope_heat_trans = xr.DataArray(cross_slope_heat_trans, coords = [('lon_bin_midpoints', lon_bin_midpoints)])
heat_trans_across_contour = xr.DataArray(heat_trans_across_contour,coords = [('lon_along_contour',lon_along_contour)])

In [81]:
outpath = '/g/data/v45/akm157/model_data/access-om2/'+exp+'/Antarctic_cross_slope/Ant_cross_slope_heat_terms_online_'+str(isobath_depth)+'m_'+time_period+'.nc'
ds = xr.Dataset({'zonal_convergence': zonal_convergence,'cross_slope_heat_trans':cross_slope_heat_trans,
                'lon_bin_midpoints':lon_bin_midpoints,'lat_bin_midpoints':lat_bin_midpoints,
                 'bin_width':bin_width,'bin_spacing':bin_spacing,'heat_trans_across_contour':heat_trans_across_contour,
                'lon_along_contour':lon_along_contour})
ds.to_netcdf(outpath)